In [2]:
# Ignore, this code will be explained later
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path: sys.path.append(module_path)

from src import utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

In [ ]:
%reload_ext post_content
%post_content register YOUR_USER_NAME

# Using `df.groupby` to zoom in to subsets of data

Generate some data

In [3]:
calories = {
    'Donuts': 195 * 1,
    'Bacon' : 43 * 3,
    'Beer'  : 154 * 1,
    'Salad' : 535 * 1,
    'Pot Roast': 252 * 2,
    'Waffles': 82 * 3,
    'Tofu'  : 94
}

k = 9
homer_food = random.choices(['Donuts', 'Bacon', 'Beer'], k=k*3)
marge_food = random.choices(['Salad', 'Pot Roast', 'Waffles'], k=k)
lisa_food  = random.choices(['Salad', 'Tofu', 'Waffles'], k=k)

days = random.choices(list(range(1,1+int(k/3))), k= k * 3 + k + k)

food_df = pd.DataFrame({
    'day' : days,
    'name': ['Homer'] * k * 3 + ['Marge'] * k + ['Lisa'] * k,
    'food': homer_food + marge_food + lisa_food,
    'calories': list(map(lambda x: calories[x],homer_food)) + list(map(lambda x: calories[x],marge_food)) + list(map(lambda x: calories[x],lisa_food))             
}).sample(frac=1).reset_index().drop(['index'],axis='columns')

food_df

,day,name,food,calories
0,3,Homer,Donuts,195
1,2,Lisa,Tofu,94
2,2,Marge,Pot Roast,504
3,2,Marge,Pot Roast,504
4,3,Homer,Bacon,129
5,1,Lisa,Tofu,94
6,1,Homer,Beer,154
7,2,Homer,Donuts,195
8,3,Homer,Beer,154
9,1,Marge,Pot Roast,504


#### Who consumed the most number of calories?

In [5]:
food_df['calories'].max()

535

In [6]:
food_df.iloc[food_df['calories'].idxmax()]

day             1
name         Lisa
food        Salad
calories      535
Name: 12, dtype: object

#### For each person, sum the number of calories

In [7]:
food_df.groupby('name').sum()

,day,calories
name,,
Homer,54,4582
Lisa,14,2762
Marge,17,3504


In [8]:
food_df.groupby('name').sum()['calories']

name
Homer    4582
Lisa     2762
Marge    3504
Name: calories, dtype: int64

### Split -> Apply -> Combine

Most programmers will immediately recognize `groupby` is a common element from SQL. Hadley Wickham's definition is useful for those not familiar with SQL:

The `groupby` function can be throught of as a three step process:

![](images/splitapplycombine.png)

In [9]:
food_df.groupby('name')

**Exercise** What is the average number of calories consumed by each person in `food_df`?

In [ ]:
%%post_content exercise_030_150_a

#type your answer here


### Iterate over splits (not used very often)

In [16]:
for name, df in food_df.groupby('name'):
    print(name)
    print(df.head())

Homer
   day   name    food  calories
0    3  Homer  Donuts       195
4    3  Homer   Bacon       129
6    1  Homer    Beer       154
7    2  Homer  Donuts       195
8    3  Homer    Beer       154
Lisa
    day  name     food  calories
1     2  Lisa     Tofu        94
5     1  Lisa     Tofu        94
12    1  Lisa    Salad       535
14    3  Lisa  Waffles       246
20    1  Lisa    Salad       535
Marge
    day   name       food  calories
2     2  Marge  Pot Roast       504
3     2  Marge  Pot Roast       504
9     1  Marge  Pot Roast       504
17    2  Marge    Waffles       246
19    3  Marge    Waffles       246


**Example** What is the mean calorie count for each person?

In [17]:
food_df.groupby('name')['calories'].mean()

name
Homer    169.703704
Lisa     306.888889
Marge    389.333333
Name: calories, dtype: float64

**Example** How many meals is each person having?

In [18]:
food_df.groupby('name').count()

,day,food,calories
name,,,
Homer,27,27,27
Lisa,9,9,9
Marge,9,9,9


**Example** How many meals _per day_ is each person eating? **(multiple group by columns)**

In [19]:
food_df.groupby(['day', 'name']).count()

food  calories
day name                 
1   Homer     9         9
    Lisa      6         6
    Marge     2         2
2   Homer     9         9
    Lisa      1         1
    Marge     6         6
3   Homer     9         9
    Lisa      2         2
    Marge     1         1

**Example** What is the minimum and maximum number of calories consumed by each person?

In [21]:
food_df.groupby('name').aggregate(['min', 'max'])

day           food          calories     
      min max        min      max      min  max
name                                           
Homer   1   3      Bacon   Donuts      129  195
Lisa    1   3      Salad  Waffles       94  535
Marge   1   3  Pot Roast  Waffles      246  504

**Example** Same as above, but get rid of the noise

In [22]:
food_df[['name', 'calories']].groupby('name').aggregate(['min', 'max'])

calories     
           min  max
name               
Homer      129  195
Lisa        94  535
Marge      246  504

In [23]:
food_df.groupby(['day', 'name']).count()

food  calories
day name                 
1   Homer     9         9
    Lisa      6         6
    Marge     2         2
2   Homer     9         9
    Lisa      1         1
    Marge     6         6
3   Homer     9         9
    Lisa      2         2
    Marge     1         1

In [26]:
food_df.groupby(['day', 'name']).count().unstack()

food            calories           
name Homer Lisa Marge    Homer Lisa Marge
day                                      
1        9    6     2        9    6     2
2        9    1     6        9    1     6
3        9    2     1        9    2     1

In [29]:
food_df.groupby(['day', 'name']).count().T

day          1                2                3           
name     Homer Lisa Marge Homer Lisa Marge Homer Lisa Marge
food         9    6     2     9    1     6     9    2     1
calories     9    6     2     9    1     6     9    2     1